In [ ]:
import os
from app_decomposer.utils_demo import *
from pprint import pprint

In [ ]:
ts = get_job_timeseries_from_file()
#ts = ts.pop("accessPattern")
pprint(ts["volume"])
#ts = ts.pop('processCount')

In [ ]:
job1 = {'bytesRead': np.array([       6457,           0,  1090522169,  2055211225]),
        'bytesWritten': np.array([         115404800,  191795200,   0,          0]),
        'timestamp': np.array([1581607510, 1581607515, 1581607520, 1581607525])}

job2 = {'bytesRead': np.array([       2457,           8000,  0,  0]),
        'bytesWritten': np.array([         15404800,  91795200,   0,          0]),
        'timestamp': np.array([1581607520, 1581607525, 1581607530, 1581607535])}

job3 = {'bytesRead': np.array([       6457,           0,  1090522169,  2055211225]),
        'bytesWritten': np.array([         115404800,  191795200,   0,          0]),
        'timestamp': np.array([1581607530, 1581607535, 1581607540, 1581607545])}

In [ ]:

jobid=3918
dataset_path = os.path.join(dirname(dirname(os.getcwd())), "dataset_generation", "dataset_generation")
job_files, job_ids, dataset_names = list_jobs(dataset_path)
# create two lists of job_ids where only few elements are differents


print(dataset_path)
print(job_ids)
print(job_files)
csv_file = job_files[job_ids.index("52508")]
df = pd.read_csv(csv_file, index_col=0)  # read the csv file into a dataframe
# print the columns of the dataframe to check if 'operationRead' exists
print(csv_file)
get_job_timeseries_from_file(job_id=7910)


In [2]:
import sys
sys.path.append("../")
import os
import unittest
from shutil import rmtree
import urllib3
from loguru import logger
from pydantic import BaseModel
from typing import List
from app_decomposer.connector import Connector, APIConnector, ApiModel
from app_decomposer.config_parser import Configuration
import unittest
import numpy as np
import pandas as pd
from unittest.mock import MagicMock
from app_decomposer.central_job import WorkflowSynthesizer, CentralJob, WorkflowSearcher, display_features_3d

CONFIG_FILE = os.path.join(os.path.dirname(os.getcwd()), "tests_integration",
                      "test_data", "test_docker_config.yaml")

In [3]:
config = Configuration(path=CONFIG_FILE)
keycloak_token = config.get_kc_token_docker()
connector = APIConnector(api_uri="http://localhost/pybackend/",
                              api_token=f"Bearer {keycloak_token}")

connector.check_connection()

False

In [4]:
searcher = WorkflowSearcher(connector)
df = searcher.search_workflows("Cryo_EM")
df.head()

2023-07-12 15:33:07.631 | DEBUG    | app_decomposer.connector:request_delegator:167 - Sending POST request to endpoint /ioi/workflows/


In [4]:
# Extract workflows data
workflows = [searcher.extract_workflow_data(workflow_id) for workflow_id in df['id']]


2023-07-12 15:29:55.011 | DEBUG    | app_decomposer.connector:request_delegator:167 - Sending GET request to endpoint /ioi/series/workflow/642b5fd8547fb888bac488ed


KeyboardInterrupt: 

In [1]:
# Convert each workflow data into the format expected by CentralJob
central_jobs = []
for workflow in workflows:
    job = {
        'bytesRead': np.array(workflow['bytesRead']),
        'bytesWritten': np.array(workflow['bytesWritten'])
    }
    central_jobs.append(job)

# Use the CentralJob class to find the central job index for each workflow
central_job = CentralJob(central_jobs)
central_job_index = central_job.find_central_job()

# Print the index of the central job
print('Central job index:', central_job_index)
# Extract corresponding line from df
central_workflow = df.iloc[central_job_index]
print(central_workflow)

NameError: name 'workflows' is not defined

In [7]:
fig = display_features_3d(central_job.features)
fig.show()

In [13]:
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances
import plotly.graph_objects as go
import numpy as np

# Apply PCA to reduce the features to 3 dimensions
pca = PCA(n_components=3)
reduced_features = pca.fit_transform(central_job.features)

# Calculate the amount of variance captured by each principal component
explained_variance = pca.explained_variance_ratio_

# Calculate the total amount of variance captured
total_variance = np.sum(explained_variance)

print(f"Variance captured by each principal component: {explained_variance}")
print(f"Total variance captured: {total_variance}")
# Calculate the Euclidean distance from each job to the centroid
distances = euclidean_distances(reduced_features, [centroid])

# Find the index of the job that is closest to the centroid
closest_job_idx = np.argmin(distances)

# Create the figure
fig = go.Figure()

# For each set of reduced features, create a scatter3d trace and add it to the figure
for i, feature in enumerate(reduced_features):
    # Choose different color and size for the closest job
    if i == closest_job_idx:
        color = 'red'
        size = 15  # Bigger size for the closest job
        name = f'Job {i} (Closest to Centroid)'
    else:
        color = i
        size = 12  # Normal size for other jobs
        name = f'Job {i}'

    fig.add_trace(go.Scatter3d(
        x=[feature[0]],
        y=[feature[1]],
        z=[feature[2]],
        mode='markers',
        marker=dict(
            size=size,
            color=color,                
            colorscale='Viridis',   
            opacity=0.8,
        ),
        text=[f'Job {i}'],
        name=name
    ))

# Add the centroid to the plot
fig.add_trace(go.Scatter3d(
    x=[centroid[0]],
    y=[centroid[1]],
    z=[centroid[2]],
    mode='markers',
    marker=dict(
        size=12,
        color='black',   # set color to black
        opacity=0.8,
    ),
    text=['Centroid'],
    name='Centroid'
))

# Set the layout properties
fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=0),
    scene=dict(
        xaxis=dict(title='PC1'),
        yaxis=dict(title='PC2'),
        zaxis=dict(title='PC3')
    )
)

# Show the plot
fig.show()
